# CSCI 3202, Fall 2025
### Homework 7
### 100 Points
### Due: October 24 by 11:59 pm

<br> 

### Your name: Rey Stone

<br> 

---

# Mancala Game Implementation

In this assignment, you are tasked with implementing various functions for a Mancala game. The game is played on a board with specific rules, and you will need to implement the core game logic by completing the `play`, `valid_move`, and `winning_eval` functions. You are provided with the `init` and `display_board` functions. The assignment is divided into two parts:

## Mancala rules for this homework assignment
**(there are many different rules sets for Mancala.  Please read this before writing the code)**

* Players sit on opposite sides of the long edge of the board
* There are 6 small pits in the middle of the board and 2 large ones at each end.  The small ones in the middle and the large pit on your right are yours.  The small ones on the other side and the large pit to your opponent's right are theirs
* The large pits at the end of the board are called Mancalas
* Set up the board with 4 stones per small pit (none in the mancalas)
* On every turn, select a pit on your side of the board that contains one or more stones,  then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
* If you encounter your opponent's mandala, skip it
* If you encounter your mancala, drop a stone into it
* If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
* If either player's pits are entirely empty, the game concludes. 
* The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.


**Please make sure to call the `display_board` function after each move for both the parts and run all the cells before submitting**

In [1]:
import random
random.seed(109)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
# re-importing here because it doesn't like me
import random
random.seed(109)

class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)  # Initialize each pit with stones_per_pit number of stones
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 1 - 1]
        self.p2_mancala_index = len(self.board) - 1

        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i],
                                                       player_2_pits[-(i + 1)], self.pits_per_player - i))
            else:
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i],
                                                       player_2_pits[-(i + 1)], self.pits_per_player - i))

        print('         {}         '.format(player_1_mancala))
        turn = color.RED + 'Turn: P1' + color.END if self.current_player == 1 else color.CYAN + 'Turn: P2' + color.END
        print(turn)

    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """

        # write your code here
        return True if pit in range(0, len(self.board)) and self.board[pit] != 0 else False

    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """

        # write your code here
        if self.current_player == 1:
            pits = range(self.p1_pits_index[0], self.p1_pits_index[1]+1)
        else:
            pits = range(self.p2_pits_index[0], self.p2_pits_index[1]+1)

        available_pits = [x for x in pits if self.board[x] > 0]

        if not available_pits:
            print('No available pits')

        chosen_pit = random.choice(available_pits)

        if self.current_player == 1:
            pit_num = chosen_pit + 1
        else:
            pit_num = chosen_pit - self.p1_mancala_index

        self.play(pit_num)


    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """

        # write your code here

        pit_index = self.get_pit_index(pit)

        if self.current_player == 1:
            print(f'{color.RED}Player {self.current_player} chose pit: {pit} {color.END}')
        else:
            print(f'{color.CYAN}Player {self.current_player} chose pit: {pit} {color.END}')

        if not self.valid_move(pit_index):
            print("INVALID MOVE")
            self.switch_player()
            return
        self.moves.append((self.current_player, pit))

        stones = self.board[pit_index]  # stones in pit
        self.board[pit_index] = 0  # empty current pit

        index = pit_index
        while stones > 0:
            index = (index + 1) % len(self.board)

            # skip over opponent's mancala
            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue

            self.board[index] += 1
            stones -= 1

        last_pit = index

        if self.current_player == 1 and self.p1_pits_index[0] <= last_pit <= self.p1_pits_index[1] :
            if self.board[last_pit] == 1: # if pit was empty before, and now it has one
                opp_pit = len(self.board) - 2 - last_pit

                if self.board[last_pit] > 0:
                    captured_stones = self.board[last_pit] + self.board[opp_pit]
                    self.board[last_pit] = 0
                    self.board[opp_pit] = 0
                    self.board[self.p1_mancala_index] = captured_stones

        if self.current_player == 2 and self.p2_pits_index[0] <= last_pit <= self.p2_pits_index[1] :
            if self.board[last_pit] == 1: # if pit was empty before, and now it has one
                opp_pit = len(self.board) - 2 - last_pit
                if self.board[last_pit] > 0:
                    captured_stones = self.board[last_pit] + self.board[opp_pit]
                    self.board[last_pit] = 0
                    self.board[opp_pit] = 0
                    self.board[self.p2_mancala_index] = captured_stones
        if self.winning_eval():
            print("GAME OVER")
            if (self.p1_mancala_index > self.p2_mancala_index):
                print("P1 win")
            elif(self.p2_mancala_index > self.p1_mancala_index):
                print("P2 win")
            else:
                print("Draw")
            return
        self.switch_player()

    # helper function to get the pit index
    def get_pit_index(self, pit_num):
        if self.current_player == 1:
            return pit_num - 1
        else:
            return self.p1_mancala_index + pit_num

    # helper function to switch the players more easily
    def switch_player(self):
        self.current_player = 2 if self.current_player == 1 else 1


    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """

        # write your code here
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1]+1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1]+1))

        return p1_empty or p2_empty



#### Output for one random agent

In [3]:
# Mancala part 2
game = Mancala(pits_per_player=6, stones_per_pit=4)
game.display_board()

# turn 1
game.play(3)
game.display_board()

game.random_move_generator()
game.display_board()

# turn 2
game.play(6)
game.display_board()

game.random_move_generator()
game.display_board()

# turn 3
game.play(5)
game.display_board()

game.random_move_generator()
game.display_board()

# turn 4
game.play(2)
game.display_board()

game.random_move_generator()
game.display_board()

# turn 5
game.play(1)
game.display_board()

game.random_move_generator()
game.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
Player 1 chose pit: 3 
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 0 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P2
Player 2 chose pit: 3 
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 4 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P1
Player 1 chose pit: 6 
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 4 | 5 | <- 5
3 -> | 0 | 6 | <- 4
4 -> | 5 | 1 | <- 3
5 -> | 5 | 5 | <- 2
6 -> |_0_|_5_| <- 1
         2         
Turn: P2
Player 2 chose pit: 2 
P1               P2
     ____2____     
1 -> | 4 | 6 | <- 6
2 -> | 4 | 6 | <- 5
3 -> | 0 | 7 | <- 4
4 -> | 5 | 2 | <- 3
5 -> | 5 | 0 | <- 2
6 -> |_0_|_5

#### Game against random agent

In [11]:
turns_per_game = []


for play_game in range(2):
    game = Mancala(pits_per_player=6, stones_per_pit=4)
    game.display_board()
    i = 0
    
    print(f"{color.BOLD + color.UNDERLINE}START GAME #: {play_game+1}{color.END}")

    while not game.winning_eval():
        print(f"{color.BOLD}Turn number: {i+1}{color.END}")

        # player 
        game.random_move_generator()
        game.display_board()

        game.random_move_generator()
        game.display_board()

        i += 1

        if game.winning_eval():
            turns_per_game.append(i)
            print(turns_per_game)

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
START GAME #: 1
Turn number: 1
Player 1 chose pit: 3 
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 0 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P2
Player 2 chose pit: 6 
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 1 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P1
Turn number: 2
Player 1 chose pit: 3 
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 0 | 4 | <- 4
4 -> | 6 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         1         
Turn: P2
Player 2 chose pit: 1 
P1               P2
     ____1____     
1 -> | 5 | 0 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> |

IndexError: Cannot choose from an empty sequence

In [ ]:
import random

class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]
        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        if pit < 1 or pit > self.pits_per_player:
            return False
        if self.current_player == 1:
            board_index = pit - 1
            if board_index < self.p1_pits_index[0] or board_index > self.p1_pits_index[1]:
                return False
            if self.board[board_index] == 0:
                return False
        else:
            board_index = self.p2_pits_index[0] + (pit - 1)
            if board_index < self.p2_pits_index[0] or board_index > self.p2_pits_index[1]:
                return False
            if self.board[board_index] == 0:
                return False
        return True
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            for pit in range(1, self.pits_per_player + 1):
                board_index = pit - 1
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        else:
            for pit in range(1, self.pits_per_player + 1):
                board_index = self.p2_pits_index[0] + (pit - 1)
                if self.board[board_index] > 0:
                    valid_pits.append(pit)
        if valid_pits:
            return random.choice(valid_pits)
        return None
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        if self.winning_eval():
            print("GAME OVER")
            return self.board
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        self.moves.append((self.current_player, pit))
        if self.current_player == 1:
            current_pit_index = pit - 1
        else:
            current_pit_index = self.p2_pits_index[0] + (pit - 1)
        stones = self.board[current_pit_index]
        self.board[current_pit_index] = 0
        current_index = current_pit_index
        opponent_mancala = self.p2_mancala_index if self.current_player == 1 else self.p1_mancala_index
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:
                continue
            self.board[current_index] += 1
            stones -= 1
        if self.current_player == 1:
            if (current_index >= self.p1_pits_index[0] and 
                current_index <= self.p1_pits_index[1] and 
                self.board[current_index] == 1):
                opposite_index = self.p2_pits_index[1] - (current_index - self.p1_pits_index[0])
                if self.board[opposite_index] > 0:
                    captured = self.board[opposite_index] + self.board[current_index]
                    self.board[opposite_index] = 0
                    self.board[current_index] = 0
                    self.board[self.p1_mancala_index] += captured
        else:
            if (current_index >= self.p2_pits_index[0] and 
                current_index <= self.p2_pits_index[1] and 
                self.board[current_index] == 1):
                opposite_index = self.p1_pits_index[1] - (current_index - self.p2_pits_index[0])
                if self.board[opposite_index] > 0:
                    captured = self.board[opposite_index] + self.board[current_index]
                    self.board[opposite_index] = 0
                    self.board[current_index] = 0
                    self.board[self.p2_mancala_index] += captured
        self.current_player = 2 if self.current_player == 1 else 1
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        if p1_empty or p2_empty:
            if not p1_empty:
                for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                    self.board[self.p1_mancala_index] += self.board[i]
                    self.board[i] = 0
            if not p2_empty:
                for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                    self.board[self.p2_mancala_index] += self.board[i]
                    self.board[i] = 0
            return True
        return False